In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from utils import extract_basic_features

import wfdb
import os
import wfdb.processing as wp
import matplotlib.pyplot as plt
from scipy import signal
from utils import find_noise_features, extract_basic_features
import shutil

import time
from lightgbm import LGBMClassifier
from scipy import sparse
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
labels = pd.read_csv("reference.csv")
labels.head()

,File_name,label1,label2,label3,label4,label5,label6,label7,label8
0,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN0003,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN0004,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAIN0005,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
sum1_label=0
sum2_label=0
sum3_label=0
sum4_label=0
sum5_label=0
sum6_label=0
sum7_label=0
bin_label = np.zeros((6500,9))
for i in range(labels.shape[0]):
    label_nona = labels.loc[i].dropna()
    if label_nona.shape[0] == 2:
        sum1_label+=1
    if label_nona.shape[0] == 3:
        sum2_label+=1
    if label_nona.shape[0] == 4:
        sum3_label+=1
    if label_nona.shape[0] == 5:
        sum4_label+=1
        print(i)
    if label_nona.shape[0] == 6:
        sum5_label+=1
    if label_nona.shape[0] == 7:
        sum6_label+=1
    if label_nona.shape[0] == 8:
        sum7_label+=1
    for j in range(1,label_nona.shape[0]):
        bin_label[i,int(label_nona[j])]=1

1896
1972
2087
2729
2977
3090
3141
3259
4227


In [4]:
labels.iloc[:2]

,File_name,label1,label2,label3,label4,label5,label6,label7,label8
0,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
num=8
column_name = ['File_name','label1','label2','label3','label4','label5','label6','label7','label8']
temp =pd.DataFrame(columns=column_name,index=range(labels.shape[0]))
temp['File_name'] = labels['File_name']
label_pd = labels[column_name].where(labels[column_name] == num,temp).dropna(axis=0,how='all')
labels_pd = label_pd.dropna(axis=0,thresh=2)
labels_pd['label1'] = labels_pd.shape[0]*[1]
labels_pd.dropna(axis=1,inplace=True)

labels_pd.columns = ['File_name1', 'label1']
labels_pd.shape

(2156, 2)

In [257]:
labels_pd = pd.concat([temp,labels_pd],axis=1)
labels_pd = labels_pd.dropna(axis=1,how='all')
#labels_pd = labels_pd['label1'].fillna(9)
#labels_pd#.dropna(subset=['File_name1'],axis=1)
labels_pd =labels_pd.drop('File_name1',axis=1)
#labels_pd.drop()
labels_pd =labels_pd.fillna(0)
labels_pd.tail()

,File_name,label1
6495,TRAIN6496,0.0
6496,TRAIN6497,0.0
6497,TRAIN6498,0.0
6498,TRAIN6499,0.0
6499,TRAIN6500,0.0


In [8]:
#2156+224+672+654+180+826+534+504+1953

7703

In [100]:
import scipy.io as sio
train_dataset_path = os.getcwd()+"/Train/"
val_dataset_path = os.getcwd()+"/Val/"

train_files = os.listdir(train_dataset_path)
train_files.sort()
val_files = os.listdir(val_dataset_path)
val_files.sort()

In [101]:
if os.path.exists("features315_II.csv"):
    features315_pd = pd.read_csv("features315_II.csv")
else:
    features315_pd = generate_features315(train_dataset_path,"TRAIN0001")
    for file_name in tqdm(train_files[1:]):
        df_temp = generate_features315(train_dataset_path,file_name.strip('.mat'))
        features315_pd = pd.concat([features315_pd,df_temp])
    features315_pd.to_csv("features315_II.csv",index=False)

In [145]:
features315_pd.head()

,File_name,bad_template_corr_coeff_median,bad_template_corr_coeff_std,detection_success,diff2_rri_activity,diff2_rri_approximate_entropy,diff2_rri_complexity,diff2_rri_fisher_info,diff2_rri_hurst_exponent,diff2_rri_kurtosis,...,template_swt_d_1_med_power_ratio,template_swt_d_2_high_power_ratio,template_swt_d_2_low_power_ratio,template_swt_d_2_med_power_ratio,template_swt_d_3_high_power_ratio,template_swt_d_3_low_power_ratio,template_swt_d_3_med_power_ratio,template_swt_d_4_high_power_ratio,template_swt_d_4_low_power_ratio,template_swt_d_4_med_power_ratio
0,TRAIN0001,0.0,0.0,0.900000,0.000328,-0.625503,1.576504,0.132745,1.075101,-1.482883,...,0.100269,0.102243,0.692252,0.092260,0.118206,0.707815,0.115713,0.127571,0.711862,0.123688
1,TRAIN0002,0.0,0.0,1.000000,0.002695,-0.625503,1.966502,0.210283,1.075101,-1.074280,...,0.082910,0.123768,0.664830,0.134348,0.100891,0.680649,0.201963,0.106492,0.684477,0.220513
2,TRAIN0003,0.0,0.0,0.866667,0.000049,-0.392436,1.475359,0.010351,1.034580,-0.034765,...,0.094416,0.118094,0.774157,0.095045,0.099268,0.784288,0.142385,0.103483,0.786120,0.157663
3,TRAIN0004,0.0,0.0,0.866667,0.001092,-0.392436,1.750371,0.102582,1.085990,-0.733251,...,0.076792,0.111508,0.590893,0.144086,0.100823,0.609346,0.206985,0.105637,0.614405,0.224029
4,TRAIN0005,0.0,0.0,0.875000,0.000097,-0.361203,1.822272,0.158262,1.104450,-1.382307,...,0.095108,0.096210,0.652023,0.089770,0.115775,0.668152,0.115918,0.126150,0.672497,0.131260


In [258]:
import lightgbm as lgb
lgb_params2={
 'learning_rate' : 0.01,
 'n_estimators':819,
 'max_depth':8,
 'num_leaves':115,
 'min_child_weight':0,
 'subsample':0.8,
 'colsample_bytree':0.8,
 'min_child_samples':21,
 'objective':'binary',
 'reg_alpha':0.15,
 'reg_lambda' : 0.01,
 'num_class': 1,
 'n_jobs':4,
 #class_weight ="1",
 'random_state' :27}

In [259]:
features315_name = set(features315_pd.columns.values) - set(['File_name'])
len(features315_name)

314

In [260]:
train_df = features315_pd#features39_pd
feature_name = features315_name

In [261]:
labels_pd["label1"].value_counts()

0.0    4344
1.0    2156
Name: label1, dtype: int64

In [262]:
train_labels = labels_pd["label1"].values

In [263]:
cv_pred_all = 0
en_amount = 1
for seed in range(en_amount):
    print("************************")
    NFOLDS = 10
    train_label = train_labels#train_data['score']
    train_data_df = train_df[feature_name].astype('float64') #feature_name  columns
    kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
    kf = kfold.split(train_data_df, train_label)

    #train_data_use = train_data.drop(['uid','score','blk_list_flag'], axis=1)
    #test_data_use = test_data.drop(['uid','blk_list_flag'], axis=1)
    train_data_use = train_data_df
    # cv_pred = np.zeros(test_data.shape[0])
    valid_best_l2_all = 0
    oof = np.zeros(train_data_df.shape[0])
    feature_importance_df = pd.DataFrame()
    count = 0
    for i, (train_fold, validate) in enumerate(kf):
        print('fold: ',i, ' training')
        X_train, X_validate, label_train, label_validate = \
        train_data_use.iloc[train_fold, :], train_data_use.iloc[validate, :], \
        train_label[train_fold], train_label[validate]
        #print(X_validate)
        #break
        dtrain = lgb.Dataset(X_train, label_train)
        dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain)
        bst = lgb.train(lgb_params2, dtrain, num_boost_round=819, valid_sets=dvalid, verbose_eval=200,early_stopping_rounds=500)
        #cv_pred += bst.predict(test_data_use, num_iteration=bst.best_iteration)
        #valid_best_l2_all += bst.best_score['valid_0']['l1']
        k_pred = bst.predict(X_validate, num_iteration=bst.best_iteration)
        #print(k_pred)
        oof[validate] = [round(i) for i in k_pred]#np.argmax(k_pred,axis=1)
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = list(X_train.columns)
        fold_importance_df["importance"] = bst.feature_importance(importance_type='split', iteration=bst.best_iteration)
        fold_importance_df["fold"] = count + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        count += 1

oof = list(map(int, oof))
fold_f1_error = f1_score(train_labels, oof, average='macro')
print('fold f1 score is {0}'.format(fold_f1_error))

from sklearn.metrics import accuracy_score
fold_acc_error = accuracy_score(train_labels, oof)
print('fold acc score is {0}'.format(fold_acc_error))


************************
fold:  0  training
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's binary_logloss: 0.381315
[400]	valid_0's binary_logloss: 0.356224
[600]	valid_0's binary_logloss: 0.35873
[800]	valid_0's binary_logloss: 0.367165
Did not meet early stopping. Best iteration is:
[448]	valid_0's binary_logloss: 0.355348
fold:  1  training
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's binary_logloss: 0.397943
[400]	valid_0's binary_logloss: 0.38387
[600]	valid_0's binary_logloss: 0.38856
[800]	valid_0's binary_logloss: 0.395279
Did not meet early stopping. Best iteration is:
[391]	valid_0's binary_logloss: 0.383233
fold:  2  training
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's binary_logloss: 0.377767
[400]	valid_0's binary_logloss: 0.35837
[600]	valid_0's binary_logloss: 0.356414
[800]	valid_0's binary_logloss: 0.360494
Did not meet early stopping. Best iteration is:
[532]	valid_0's

In [264]:
oof = list(map(int, oof))
fold_f1_error = f1_score(train_labels, oof, average='macro')
print('fold f1 score is {0}'.format(fold_f1_error))

from sklearn.metrics import accuracy_score
fold_acc_error = accuracy_score(train_labels, oof)
print('fold acc score is {0}'.format(fold_acc_error))


fold f1 score is 0.8097634373830693
fold acc score is 0.8369230769230769


In [213]:
#0 - Normal; 0.86  0.89
#1 - AF; 0.96  0.99
#2 - FDAVB; 0.83 0.95
#3 - CRBBB; 0.83  0.93
#4 - LAFB; 0.787  0.98
#5 - PVC; 0.91  0.97
#6 - PAC; 0.92  0.97
#7 - ER; 0.58   0.96
#8 - TWC 0.80  0.83

In [265]:
bst.save_model("lightgbm_2_class_TWC.bin")

In [217]:
if os.path.exists("val_features315_II.csv"):
    val_features315_pd = pd.read_csv("val_features315_II.csv")
else:
    val_features315_pd = generate_features315(val_dataset_path,"VAL001")
    for file_name in tqdm(val_files[1:]):
        df_temp = generate_features315(val_dataset_path,file_name.strip('.mat'))
        val_features315_pd = pd.concat([val_features315_pd,df_temp])
    val_features315_pd.to_csv("val_features315_II.csv",index=False)

In [238]:
kpred = []

In [266]:
import lightgbm as lgb 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
model_path = './two_class_model/'
bst = lgb.Booster(model_file=model_path+'lightgbm_2_class_TWC.bin')

In [267]:
dfeatures = val_features315_pd[feature_name].values
pred = bst.predict(dfeatures)
kpred.append([int(round(i)) for i in pred])
len(kpred)

9

In [272]:
pred = np.array(kpred).T

In [275]:
pred[:20]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [285]:
an_pred = []
for i in range(pred.shape[0]):
    an_pred.append(list(np.hstack(np.where(pred[i]==1))))

In [287]:
import csv
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = an_pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()

In [ ]:
import csv

dfeatures = val_features315_pd[feature_name].values
kpred = bst.predict(dfeatures)

pred = []
for i in range(kpred.shape[0]):
    pred.append( list(np.hstack(np.argwhere(kpred[i]>0.15))))
    
with open('answers.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                    'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):
            
            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)
            
            result = pred[count]
            
            answer.extend(result)
            for i in range(8-len(result)):
                answer.append('')
                
            #print(answer)
            count += 1
            writer.writerow(answer)
    csvfile.close()